## Game Data: Basketball Reference

The site [www.basketball-reference.com](https://www.basketball-reference.com/leagues/NBA_2019_games.html) has a massive amount of statistics that can be used in our modeling, i.e. a machine learning model to predict the outcome of a game.

## Betting Data: ScoresAndOdds and Sportsbook Review

The site [www.scoresandodds.com](https://www.scoresandodds.com/) has betting data that we can incorporate into the analysis. Though I thought this was a great resource at first, I found it only had data from 2018-01-01 to present, but then I came across [Sportsbook Review](https://www.sportsbookreview.com/). This site has data that goes back further and incorporates many individual betting sites.

## My Database

I have already scraped a pretty large amount of data from all three websites. This can be queried directly into Python:

In [10]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="localhost",
    database="sports",
    user="postgres",
    password=open("core/password").read()
)
cursor = conn.cursor()
# I put the basketball-reference data in the "games" table
cursor.execute("select * from games")
games = pd.DataFrame(cursor.fetchall())
games.columns = [i[0] for i in cursor.description]
cursor.close()
conn.commit()

games.head()

,code,season,game_date,start_time,home_team,home_code,home_points,visiting_team,visiting_code,visitor_points,has_ot,attendance,winner
0,200102010CLE,2001,2001-02-01,19:30:00,Cleveland Cavaliers,CLE,81,Minnesota Timberwolves,MIN,90,False,13904,MIN
1,200102010DAL,2001,2001-02-01,20:00:00,Dallas Mavericks,DAL,95,Miami Heat,MIA,91,True,14497,DAL
2,200102010NYK,2001,2001-02-01,20:00:00,New York Knicks,NYK,80,Philadelphia 76ers,PHI,87,False,19763,PHI
3,200102010HOU,2001,2001-02-01,20:30:00,Houston Rockets,HOU,84,Los Angeles Clippers,LAC,101,False,11124,LAC
4,200102010UTA,2001,2001-02-01,21:00:00,Utah Jazz,UTA,87,Charlotte Hornets,CHH,76,False,18855,UTA


Here's an example of the sports betting data:

In [27]:
cursor = conn.cursor()
cursor.execute("select * from line_movements where home != 'Off'")
odds = pd.DataFrame(cursor.fetchall())
odds.columns = [i[0] for i in cursor.description]
cursor.close()
conn.commit()

odds.head()

,key,date_timestamp,date_url,home_team,away_team,home,away
0,179454,2018-10-07 19:05:00,2018-10-07,Oklahoma City Thunder,Atlanta Hawks,-253,+206
1,179455,2018-10-07 20:05:00,2018-10-07,San Antonio Spurs,Houston Rockets,-122,+103
2,179456,2018-10-08 00:05:00,2018-10-07,Minnesota Timberwolves,Milwaukee Bucks,+110,-131
3,179457,2018-10-08 01:05:00,2018-10-07,Portland Trail Blazers,Utah Jazz,+128,-152
4,179477,2018-10-10 00:05:00,2018-10-09,Oklahoma City Thunder,Milwaukee Bucks,-216,+178


These can easily be converted to probabilities:

In [32]:
def get_implied_probability(ml):
    if ml > 0:
        return 100 / (ml + 100)
    else:
        return -ml / (-ml + 100)

odds["home"] = odds["home"].astype(float)
odds["home_probability"] = odds["home"].apply(get_implied_probability)
odds["away_probability"] = 1 - odds["home_probability"]
odds.head()

,key,date_timestamp,date_url,home_team,away_team,home,away,home_probability,away_probability
0,179454,2018-10-07 19:05:00,2018-10-07,Oklahoma City Thunder,Atlanta Hawks,-253.0,+206,0.716714,0.283286
1,179455,2018-10-07 20:05:00,2018-10-07,San Antonio Spurs,Houston Rockets,-122.0,+103,0.549550,0.450450
2,179456,2018-10-08 00:05:00,2018-10-07,Minnesota Timberwolves,Milwaukee Bucks,110.0,-131,0.476190,0.523810
3,179457,2018-10-08 01:05:00,2018-10-07,Portland Trail Blazers,Utah Jazz,128.0,-152,0.438596,0.561404
4,179477,2018-10-10 00:05:00,2018-10-09,Oklahoma City Thunder,Milwaukee Bucks,-216.0,+178,0.683544,0.316456


## Exploratory Data Analysis

One obvious question: how good are the odds at predicting the outcome of a game? I join together the two tables and find that the odds are correct roughly 67% of the time.

In [34]:
query = """
with main as (
  select g.*
  from games g
  where home != 'Off '
  order by home_team, game_date
),

line_moves as (
  select * from line_movements lm
)

select 
  mn1.*,
  case when mn1.home_code = winner then 1 else 0 end as home_team_wins,
  lm.home as money_line
from main mn1
  inner join line_moves lm on mn1.game_date = date_url and mn1.home_team = lm.home_team and mn1.visiting_team = lm.away_team
"""

cursor = conn.cursor()
cursor.execute(query)
joined = pd.DataFrame(cursor.fetchall())
joined.columns = [i[0] for i in cursor.description]
cursor.close()
conn.commit()

In [44]:
import numpy as np
from sklearn.metrics import confusion_matrix,  accuracy_score

joined["money_line"] = joined["money_line"].astype(float)
joined["money_line_predict"] = np.where(joined["money_line"] < 0, 1, 0)

In [45]:
accuracy_score(joined["home_team_wins"], joined["money_line_predict"])

0.6730462519936204

In [46]:
confusion_matrix(joined["home_team_wins"], joined["money_line_predict"])

array([[250, 262],
       [148, 594]])